In [1]:
# Trip Advisor AI Agent using LangChain + Ollama (Local Llama3)
# Import required libraries
import os
from datetime import datetime
from typing import List, Dict, Any, Optional

# LangChain imports
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from langchain.tools import BaseTool
from langchain.callbacks.manager import CallbackManagerForToolRun
from langchain_ollama import OllamaLLM
from pydantic import BaseModel, Field

print("Libraries imported successfully!")
print("Ready to build your Trip Advisor AI Agent with Local Llama3!")

Libraries imported successfully!
Ready to build your Trip Advisor AI Agent with Local Llama3!


In [2]:
# Configuration and Ollama Setup
# Check if Ollama is running and has Llama3 model
import subprocess
import requests
import json

def check_ollama_status():
    """Check if Ollama is running and has Llama3 model available"""
    try:
        # Check if Ollama server is running
        response = requests.get("http://localhost:11434/api/tags", timeout=5)
        if response.status_code == 200:
            models = response.json()
            model_names = [model['name'] for model in models.get('models', [])]
            
            # Check for Llama3 variants
            llama3_models = [name for name in model_names if 'llama3' in name.lower()]
            
            if llama3_models:
                print("✅ Ollama is running!")
                print(f"✅ Available Llama3 models: {llama3_models}")
                return True, llama3_models[0]  # Return first available Llama3 model
            else:
                print("⚠️  Ollama is running but no Llama3 model found.")
                print("Available models:", model_names)
                print("\n📥 To install Llama3, run: ollama pull llama3")
                return False, None
        else:
            print("❌ Ollama server responded with error")
            return False, None
    except requests.exceptions.ConnectionError:
        print("❌ Ollama server is not running!")
        print("\n🚀 To start Ollama:")
        print("1. Install Ollama from: https://ollama.ai/")
        print("2. Run: ollama serve")
        print("3. Pull Llama3: ollama pull llama3")
        return False, None
    except Exception as e:
        print(f"❌ Error checking Ollama: {e}")
        return False, None

# Check Ollama status
ollama_available, llama3_model = check_ollama_status()

✅ Ollama is running!
✅ Available Llama3 models: ['llama3:latest', 'llama3-8b-instruct:latest']


In [3]:
# Custom Tools for Trip Planning
class DestinationRecommenderTool(BaseTool):
    """Tool to recommend destinations based on preferences"""
    name: str = "destination_recommender"
    description: str = "Recommends travel destinations based on user preferences like budget, interests, season, etc."
    
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        # This is a simplified implementation - in reality, you'd connect to a travel database or API
        preferences = query.lower()
        
        recommendations = []
        
        if "beach" in preferences or "tropical" in preferences:
            recommendations.extend(["Maldives", "Bali", "Caribbean Islands", "Hawaii"])
        if "mountain" in preferences or "hiking" in preferences:
            recommendations.extend(["Swiss Alps", "Nepal", "Patagonia", "Rocky Mountains"])
        if "culture" in preferences or "history" in preferences:
            recommendations.extend(["Rome", "Kyoto", "Istanbul", "Cairo"])
        if "budget" in preferences or "cheap" in preferences:
            recommendations.extend(["Thailand", "Vietnam", "India", "Eastern Europe"])
        if "luxury" in preferences or "expensive" in preferences:
            recommendations.extend(["Monaco", "Dubai", "Tokyo", "New York"])
        if "adventure" in preferences:
            recommendations.extend(["New Zealand", "Costa Rica", "Iceland", "Patagonia"])
        
        if not recommendations:
            recommendations = ["Paris", "London", "Barcelona", "Amsterdam", "Prague"]
        
        return f"Based on your preferences '{query}', I recommend these destinations: {', '.join(recommendations[:5])}"

class ItineraryPlannerTool(BaseTool):
    """Tool to create travel itineraries"""
    name: str = "itinerary_planner"
    description: str = "Creates detailed travel itineraries for specific destinations including activities, restaurants, and timing."
    
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        # Parse the destination and duration from query
        destination = query.split(" for ")[0] if " for " in query else query
        
        # Sample itinerary data (in a real app, this would come from a database or API)
        itineraries = {
            "paris": {
                "day1": "Morning: Visit Eiffel Tower, Afternoon: Louvre Museum, Evening: Seine River Cruise",
                "day2": "Morning: Notre-Dame Cathedral, Afternoon: Montmartre & Sacré-Cœur, Evening: Latin Quarter dinner",
                "day3": "Morning: Palace of Versailles, Afternoon: Champs-Élysées shopping, Evening: Moulin Rouge show"
            },
            "rome": {
                "day1": "Morning: Colosseum tour, Afternoon: Roman Forum, Evening: Trastevere dinner",
                "day2": "Morning: Vatican Museums & Sistine Chapel, Afternoon: St. Peter's Basilica, Evening: Spanish Steps",
                "day3": "Morning: Pantheon, Afternoon: Trevi Fountain & Villa Borghese, Evening: Campo de' Fiori"
            },
            "tokyo": {
                "day1": "Morning: Senso-ji Temple, Afternoon: Tokyo Skytree, Evening: Shibuya Crossing",
                "day2": "Morning: Tsukiji Fish Market, Afternoon: Imperial Palace, Evening: Ginza district",
                "day3": "Morning: Meiji Shrine, Afternoon: Harajuku & Omotesando, Evening: Robot Restaurant"
            }
        }
        
        dest_key = destination.lower().replace(" ", "")
        if dest_key in itineraries:
            itinerary = itineraries[dest_key]
            result = f"Here's a 3-day itinerary for {destination}:\n\n"
            for day, activities in itinerary.items():
                result += f"{day.upper()}: {activities}\n"
            return result
        else:
            return f"I don't have a specific itinerary for {destination} yet, but I can help you plan based on popular attractions and activities there!"

class BudgetCalculatorTool(BaseTool):
    """Tool to estimate travel costs"""
    name: str = "budget_calculator"
    description: str = "Calculates estimated travel costs including flights, accommodation, food, and activities."
    
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        # Sample budget estimates (in reality, you'd use real-time pricing APIs)
        budgets = {
            "paris": {"flights": 800, "hotel": 150, "food": 60, "activities": 40},
            "rome": {"flights": 700, "hotel": 120, "food": 45, "activities": 35},
            "tokyo": {"flights": 1200, "hotel": 180, "food": 80, "activities": 50},
            "thailand": {"flights": 900, "hotel": 40, "food": 15, "activities": 25},
            "new york": {"flights": 500, "hotel": 250, "food": 70, "activities": 60}
        }
        
        destination = query.lower().replace(" ", "")
        days = 7  # Default to 7 days
        
        # Try to extract number of days from query
        import re
        day_match = re.search(r'(\d+)\s*days?', query.lower())
        if day_match:
            days = int(day_match.group(1))
        
        if destination in budgets:
            budget = budgets[destination]
            total_per_day = budget["hotel"] + budget["food"] + budget["activities"]
            total_cost = budget["flights"] + (total_per_day * days)
            
            return f"""Budget estimate for {destination.title()} ({days} days):
            
Flight: ${budget['flights']}
Hotel: ${budget['hotel']}/night × {days} nights = ${budget['hotel'] * days}
Food: ${budget['food']}/day × {days} days = ${budget['food'] * days}
Activities: ${budget['activities']}/day × {days} days = ${budget['activities'] * days}

Total estimated cost: ${total_cost}
Daily cost (excluding flights): ${total_per_day}"""
        else:
            return f"I don't have specific budget data for {destination}, but typical costs vary widely by destination. Budget destinations: $50-100/day, Mid-range: $100-200/day, Luxury: $300+/day"

print("Custom tools created successfully!")

Custom tools created successfully!


In [4]:
# Additional Tools for comprehensive trip planning
class WeatherInfoTool(BaseTool):
    """Tool to provide weather and seasonal information"""
    name: str = "weather_info"
    description: str = "Provides weather information and best times to visit destinations."
    
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        destination = query.lower()
        
        # Sample weather data (in reality, you'd use a weather API)
        weather_data = {
            "paris": {
                "best_months": "April-June, September-October",
                "climate": "Temperate oceanic climate",
                "summer": "Warm (15-25°C), can be rainy",
                "winter": "Cool (3-7°C), occasional snow"
            },
            "tokyo": {
                "best_months": "March-May, September-November", 
                "climate": "Humid subtropical climate",
                "summer": "Hot and humid (25-30°C), rainy season June-July",
                "winter": "Cool and dry (5-10°C)"
            },
            "thailand": {
                "best_months": "November-March",
                "climate": "Tropical monsoon climate",
                "summer": "Hot (28-35°C), rainy season May-October",
                "winter": "Warm and dry (25-30°C)"
            }
        }
        
        dest_key = destination.replace(" ", "").lower()
        if dest_key in weather_data:
            data = weather_data[dest_key]
            return f"""Weather information for {destination.title()}:

Climate: {data['climate']}
Best months to visit: {data['best_months']}
Summer: {data['summer']}
Winter: {data['winter']}

Remember to check current weather forecasts before traveling!"""
        else:
            return f"I don't have specific weather data for {destination}, but I recommend checking current weather forecasts and seasonal patterns before planning your trip."

class TravelTipsTool(BaseTool):
    """Tool to provide travel tips and cultural information"""
    name: str = "travel_tips"
    description: str = "Provides travel tips, cultural etiquette, and practical advice for destinations."
    
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        destination = query.lower()
        
        tips_data = {
            "japan": [
                "Remove shoes when entering homes and some restaurants",
                "Bow as a greeting instead of shaking hands",
                "Don't tip - it's considered rude",
                "Learn basic Japanese phrases",
                "Get a JR Pass for train travel",
                "Cash is preferred over cards in many places"
            ],
            "thailand": [
                "Dress modestly when visiting temples",
                "Don't touch someone's head",
                "Remove shoes before entering temples and homes",
                "Bargain at markets but not in malls",
                "Try street food - it's usually safe and delicious",
                "Learn basic Thai greetings"
            ],
            "france": [
                "Learn basic French phrases - locals appreciate the effort",
                "Dress well, especially in Paris",
                "Greet shopkeepers when entering stores",
                "Lunch is typically 12-2pm, dinner after 7:30pm",
                "Tipping 10% is sufficient in restaurants",
                "Many museums are free on first Sunday mornings"
            ]
        }
        
        dest_key = destination.replace(" ", "").lower()
        if dest_key in tips_data:
            tips = tips_data[dest_key]
            result = f"Travel tips for {destination.title()}:\n\n"
            for i, tip in enumerate(tips, 1):
                result += f"{i}. {tip}\n"
            return result
        else:
            return f"General travel tips: Research local customs, learn basic phrases, respect dress codes, try local cuisine, and always have travel insurance!"

print("Additional tools created successfully!")

Additional tools created successfully!


In [5]:
# Initialize the AI Agent with LangChain + Ollama
from typing import Optional

# Fix imports for the tools
def create_trip_advisor_agent(model_name="llama3"):
    """Creates and returns a configured trip advisor AI agent using Ollama"""
    
    # Initialize the local Ollama language model
    llm = OllamaLLM(
        model=model_name,
        base_url="http://localhost:11434",
        temperature=0.1,  # Low temperature for consistent, factual responses
    )
    
    # Create tool instances
    tools = [
        DestinationRecommenderTool(),
        ItineraryPlannerTool(),
        BudgetCalculatorTool(),
        WeatherInfoTool(),
        TravelTipsTool()
    ]
    
    # Create memory for conversation history
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )
    
    # Create the agent with tools and memory
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        verbose=True,
        handle_parsing_errors=True
    )
    
    return agent

# Custom system prompt for the trip advisor
TRIP_ADVISOR_PROMPT = """
You are a friendly and knowledgeable AI trip advisor assistant powered by Llama3. Your role is to help users plan their perfect trips by:

1. Understanding their travel preferences, budget, and interests
2. Recommending suitable destinations
3. Creating detailed itineraries
4. Providing budget estimates
5. Offering weather information and best travel times
6. Sharing cultural tips and practical advice

Always be helpful, enthusiastic, and provide detailed, actionable advice. Use the available tools to gather information and give accurate recommendations.

When users ask questions:
- Be conversational and friendly
- Ask clarifying questions if needed
- Use the tools to provide specific, accurate information
- Give practical tips and suggestions
- Consider their budget, interests, and travel style

Remember: You're here to make travel planning easier and more enjoyable!
"""

print("Trip advisor agent setup ready with Ollama!")
print("Next: Make sure Ollama is running with Llama3 and create the agent instance.")

Trip advisor agent setup ready with Ollama!
Next: Make sure Ollama is running with Llama3 and create the agent instance.


In [6]:
# Create and Test the Agent with Ollama
def test_agent():
    """Test function to create and interact with the trip advisor agent"""
    if not ollama_available:
        print("❌ Cannot create agent - Ollama is not available!")
        print("Please ensure Ollama is running and has Llama3 installed.")
        return None
    
    try:
        # Create the agent with the available Llama3 model
        agent = create_trip_advisor_agent(llama3_model)
        print(f"✅ Trip Advisor Agent created successfully using {llama3_model}!")
        
        return agent
    except Exception as e:
        print(f"❌ Error creating agent: {e}")
        print("Make sure Ollama is running and accessible!")
        return None

# Test the agent creation (this will only work if Ollama is running)
if ollama_available:
    trip_agent = test_agent()
else:
    print("⚠️  Please start Ollama and install Llama3 first, then run this cell again.")

✅ Trip Advisor Agent created successfully using llama3:latest!


C:\Users\C00580017\AppData\Local\Temp\ipykernel_28012\609520061.py:25: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
C:\Users\C00580017\AppData\Local\Temp\ipykernel_28012\609520061.py:31: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [7]:
# Interactive Chat Function
def chat_with_agent(agent, message):
    """Function to chat with the trip advisor agent"""
    if agent is None:
        return "Please create the agent first by ensuring Ollama is running!"
    
    try:
        response = agent.run(message)
        return response
    except Exception as e:
        return f"Error: {e}"

# Example usage and test queries
def run_example_queries():
    """Run some example queries to demonstrate the agent capabilities"""
    if not ollama_available:
        print("Please start Ollama and install Llama3 first!")
        return
    
    # Sample queries to test
    test_queries = [
        "I want to visit a tropical destination for 7 days with a budget of $2000",
        "Create an itinerary for Paris for 3 days",
        "What's the weather like in Tokyo in spring?",
        "Give me travel tips for visiting Thailand",
        "Calculate budget for Rome for 5 days"
    ]
    
    print("=== Testing Trip Advisor Agent with Llama3 ===\n")
    
    for i, query in enumerate(test_queries, 1):
        print(f"Query {i}: {query}")
        print("-" * 50)
        response = chat_with_agent(trip_agent, query)
        print(f"Response: {response}\n")

# Instructions for usage
print("""
🎯 How to use your Local Trip Advisor Agent:

1. Install and start Ollama:
   - Download from: https://ollama.ai/
   - Run: ollama serve
   - Install Llama3: ollama pull llama3

2. The agent will be created automatically if Ollama is running

3. Use the chat function:
   response = chat_with_agent(trip_agent, "your question here")

4. Or run example queries:
   run_example_queries()

Example questions you can ask:
- "I want to visit Europe in summer, budget $3000"
- "Plan a 5-day itinerary for Tokyo"
- "What's the best time to visit Thailand?"
- "Give me travel tips for Japan"
- "Calculate costs for a week in Paris"

🚀 Benefits of using Ollama + Llama3:
✅ Completely free - no API costs
✅ Privacy - everything runs locally
✅ No internet required after setup
✅ Fast responses once model is loaded
""")


🎯 How to use your Local Trip Advisor Agent:

1. Install and start Ollama:
   - Download from: https://ollama.ai/
   - Run: ollama serve
   - Install Llama3: ollama pull llama3

2. The agent will be created automatically if Ollama is running

3. Use the chat function:
   response = chat_with_agent(trip_agent, "your question here")

4. Or run example queries:
   run_example_queries()

Example questions you can ask:
- "I want to visit Europe in summer, budget $3000"
- "Plan a 5-day itinerary for Tokyo"
- "What's the best time to visit Thailand?"
- "Give me travel tips for Japan"
- "Calculate costs for a week in Paris"

🚀 Benefits of using Ollama + Llama3:
✅ Completely free - no API costs
✅ Privacy - everything runs locally
✅ No internet required after setup
✅ Fast responses once model is loaded



In [8]:
# Test your Trip Advisor Agent!
# Let's ask it a question to see it in action

print("🌍 Testing your Trip Advisor Agent with a sample question...")
print("=" * 60)

# Test question
test_question = "I want to visit a tropical destination for 7 days with a budget of $2000"
print(f"Question: {test_question}")
print("-" * 60)

# Get response from the agent
response = chat_with_agent(trip_agent, test_question)
print(f"Agent Response:\n{response}")
print("=" * 60)
print("🎉 Your agent is working! Try asking your own questions now.")

🌍 Testing your Trip Advisor Agent with a sample question...
Question: I want to visit a tropical destination for 7 days with a budget of $2000
------------------------------------------------------------


> Entering new AgentExecutor chain...


C:\Users\C00580017\AppData\Local\Temp\ipykernel_28012\109982021.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(message)


```json
{
    "action": "budget_calculator",
    "action_input": "$2000"
}
```

Please provide the estimated travel costs including flights, accommodation, food, and activities based on the user's budget.
Observation: I don't have specific budget data for $2000, but typical costs vary widely by destination. Budget destinations: $50-100/day, Mid-range: $100-200/day, Luxury: $300+/day
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Budget destinations typically cost $50-100 per day, while mid-range options range from $100-200 per day, and luxury destinations can exceed $300 per day."
}
```

> Finished chain.
Agent Response:
Budget destinations typically cost $50-100 per day, while mid-range options range from $100-200 per day, and luxury destinations can exceed $300 per day.
🎉 Your agent is working! Try asking your own questions now.


In [9]:
# Let's try another test with destination recommendations
print("\n🏝️ Testing destination recommendations...")
print("=" * 60)

# Test destination recommendation
question2 = "I love beaches and tropical weather, what destinations do you recommend?"
print(f"Question: {question2}")
print("-" * 60)

response2 = chat_with_agent(trip_agent, question2)
print(f"Agent Response:\n{response2}")
print("=" * 60)


🏝️ Testing destination recommendations...
Question: I love beaches and tropical weather, what destinations do you recommend?
------------------------------------------------------------


> Entering new AgentExecutor chain...
```json
{
    "action": "destination_recommender",
    "action_input": "beach, tropical, budget $2000"
}
```

Please use the destination_recommender tool to suggest some beachy and tropical destinations that fit within your budget of $2000.
Observation: Based on your preferences 'beach, tropical, budget $2000', I recommend these destinations: Maldives, Bali, Caribbean Islands, Hawaii, Thailand
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Considering your love for beaches and tropical weather, the Maldives, Bali, Caribbean Islands, Hawaii, and Thailand are excellent options that fit within your budget of $2000."
}
```

> Finished chain.
Agent Response:
Considering your love for beaches and tropical weather, the Maldives, Bali, Caribbean Isl

In [10]:
# Test itinerary planning
print("\n📅 Testing itinerary planning...")
print("=" * 60)

question3 = "Create an itinerary for Paris for 3 days"
print(f"Question: {question3}")
print("-" * 60)

response3 = chat_with_agent(trip_agent, question3)
print(f"Agent Response:\n{response3}")
print("=" * 60)


📅 Testing itinerary planning...
Question: Create an itinerary for Paris for 3 days
------------------------------------------------------------


> Entering new AgentExecutor chain...
```json
{
    "action": "itinerary_planner",
    "action_input": "Paris, 3 days"
}
```

Please use the `itinerary_planner` tool to create a detailed travel itinerary for Paris for 3 days.
Observation: I don't have a specific itinerary for Paris, 3 days yet, but I can help you plan based on popular attractions and activities there!
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Paris is known for its iconic landmarks like the Eiffel Tower, Notre-Dame Cathedral, and Arc de Triomphe. You can spend your first day exploring the Latin Quarter, visiting the famous Shakespeare and Company bookstore, and enjoying a Seine River cruise at sunset. On the second day, visit the Louvre Museum to see the Mona Lisa and other famous artworks. In the evening, enjoy a cabaret show at the world-famous M

In [11]:
# 🗣️ Chat with your Trip Advisor Agent!
# Change the question below to ask anything about travel planning

# Your question here - change this to ask whatever you want!
your_question = "What's the weather like in Tokyo in spring?"

print(f"🤔 You asked: {your_question}")
print("🤖 Agent thinking...")
print("-" * 60)

# Get response from your agent
response = chat_with_agent(trip_agent, your_question)
print(f"Agent: {response}")

print("\n" + "="*60)
print("💡 Try changing 'your_question' above and run this cell again!")
print("💡 Example questions:")
print("   - 'Give me travel tips for Japan'")
print("   - 'Calculate budget for Rome for 5 days'")
print("   - 'I want adventure travel recommendations'")
print("   - 'Plan a romantic getaway to Bali'")
print("="*60)

🤔 You asked: What's the weather like in Tokyo in spring?
🤖 Agent thinking...
------------------------------------------------------------


> Entering new AgentExecutor chain...
```json
{
    "action": "weather_info",
    "action_input": "Tokyo, Spring"
}
```
Observation: I don't have specific weather data for tokyo, spring, but I recommend checking current weather forecasts and seasonal patterns before planning your trip.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "I don't have specific weather data for Tokyo in spring, but I recommend checking current weather forecasts and seasonal patterns before planning your trip."
}
```

> Finished chain.
Agent: I don't have specific weather data for Tokyo in spring, but I recommend checking current weather forecasts and seasonal patterns before planning your trip.

💡 Try changing 'your_question' above and run this cell again!
💡 Example questions:
   - 'Give me travel tips for Japan'
   - 'Calculate budget for Rome for 5 

In [12]:
# Optional: Create a Streamlit Web Interface
def create_streamlit_app():
    """Creates a Streamlit web interface for the trip advisor agent"""
    
    streamlit_code = '''
import streamlit as st
import os
from your_agent_code import create_trip_advisor_agent, chat_with_agent

# Streamlit App
st.title("🌍 AI Trip Advisor")
st.subheader("Your Personal Travel Planning Assistant")

# Sidebar for API key
with st.sidebar:
    st.header("Configuration")
    api_key = st.text_input("OpenAI API Key", type="password")
    if api_key:
        os.environ["OPENAI_API_KEY"] = api_key
        st.success("API Key set!")

# Initialize agent
if "agent" not in st.session_state and api_key:
    try:
        st.session_state.agent = create_trip_advisor_agent()
        st.success("Agent initialized!")
    except Exception as e:
        st.error(f"Error initializing agent: {e}")

# Chat interface
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Chat input
if prompt := st.chat_input("Ask me about your travel plans!"):
    # Add user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # Get agent response
    if "agent" in st.session_state:
        with st.chat_message("assistant"):
            with st.spinner("Planning your trip..."):
                response = chat_with_agent(st.session_state.agent, prompt)
                st.markdown(response)
                st.session_state.messages.append({"role": "assistant", "content": response})
    else:
        st.error("Please configure your OpenAI API key first!")

# Example questions
st.sidebar.header("Example Questions")
examples = [
    "I want to visit Europe in summer",
    "Plan a 5-day itinerary for Tokyo", 
    "What's the best time to visit Thailand?",
    "Calculate costs for a week in Paris"
]

for example in examples:
    if st.sidebar.button(example):
        st.session_state.messages.append({"role": "user", "content": example})
        st.rerun()
'''
    
    # Save the Streamlit app code to a file
    with open("trip_advisor_app.py", "w", encoding="utf-8") as f:
        f.write(streamlit_code)
    
    print("✅ Streamlit app code saved to 'trip_advisor_app.py'")
    print("To run the web interface:")
    print("1. Save your agent code to a separate Python file")
    print("2. Run: streamlit run trip_advisor_app.py")

# Create the Streamlit app file
create_streamlit_app()

✅ Streamlit app code saved to 'trip_advisor_app.py'
To run the web interface:
1. Save your agent code to a separate Python file
2. Run: streamlit run trip_advisor_app.py


In [ ]:
# 🎉 Your Local AI Trip Advisor Agent is Ready!

print("""
🚀 CONGRATULATIONS! Your LangChain + Ollama + Llama3 Trip Advisor Agent is complete!

📋 WHAT YOU'VE BUILT:
✅ Custom LangChain tools for trip planning
✅ Destination recommendation system
✅ Itinerary planner
✅ Budget calculator  
✅ Weather information provider
✅ Travel tips and cultural advice
✅ Conversational AI agent with memory
✅ LOCAL Llama3 model (no API costs!)
✅ Complete privacy (everything runs locally)

🔑 SETUP REQUIREMENTS:
1. Install Ollama from: https://ollama.ai/
2. Start Ollama: ollama serve
3. Pull Llama3: ollama pull llama3
4. Run all cells to create your agent
5. Start chatting with your AI trip advisor!

💡 EXAMPLE QUESTIONS TO TRY:
• "I want to visit Asia for 10 days with a $3000 budget"
• "Plan a romantic getaway to Paris for 4 days"
• "What's the best time to visit Japan for cherry blossoms?"
• "Give me travel tips for backpacking through Europe"
• "Calculate the cost for a luxury trip to Maldives"

🔧 CUSTOMIZATION IDEAS:
• Add more destinations to the tools
• Connect to real travel APIs (Booking.com, Skyscanner, etc.)
• Add restaurant recommendations
• Include visa requirements
• Add packing list generator
• Try different Ollama models (llama3:70b, mistral, etc.)

🌐 DEPLOY OPTIONS:
• Streamlit web app (included)
• Flask/FastAPI web service
• Discord/Telegram bot
• Desktop application with GUI

🎯 ADVANTAGES OF LOCAL SETUP:
✅ FREE - No API costs ever
✅ PRIVATE - Your data stays on your machine
✅ FAST - No network latency
✅ OFFLINE - Works without internet
✅ CUSTOMIZABLE - Full control over the model

Happy travels with your local AI assistant! 🌍✈️
""")